In [32]:
import sys
!{sys.executable} -m pip install --user seaborn scikit-learn pandas numpy matplotlib

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn as sk


[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [33]:
accidentdf1 = pd.read_csv('archive/accidents_2005_to_2007.csv')
accidentdf2 = pd.read_csv('archive/accidents_2009_to_2011.csv')
accidentdf3 = pd.read_csv('archive/accidents_2012_to_2014.csv')

df_concat = pd.concat([accidentdf1, accidentdf2, accidentdf3])

C:\Users\notgui\AppData\Local\Temp\ipykernel_30996\1967487325.py:1: DtypeWarning: Columns (31) have mixed types. Specify dtype option on import or set low_memory=False.
  accidentdf1 = pd.read_csv('archive/accidents_2005_to_2007.csv')
C:\Users\notgui\AppData\Local\Temp\ipykernel_30996\1967487325.py:3: DtypeWarning: Columns (31) have mixed types. Specify dtype option on import or set low_memory=False.
  accidentdf3 = pd.read_csv('archive/accidents_2012_to_2014.csv')


In [ ]:
df = df_concat.drop(columns = ['Accident_Index','Location_Easting_OSGR',
                            'Location_Northing_OSGR','Date','LSOA_of_Accident_Location',
                            'Local_Authority_(District)','Local_Authority_(Highway)',
                            '1st_Road_Number','Longitude','Latitude',
                            '1st_Road_Class', 'Road_Type',
                            'Police_Force','Junction_Detail',
                            '2nd_Road_Class','2nd_Road_Number'
                            ])

# print(df.info())
df['Junction_Control'] = df['Junction_Control'].fillna('No junction')
df['Light_Conditions'] = df['Light_Conditions'].replace(
    {'Darkeness: No street lighting': 'Darkness: No street lighting'}
)
df['Special_Conditions_at_Site'] = df['Special_Conditions_at_Site'].fillna('None')
df['Carriageway_Hazards'] = df['Carriageway_Hazards'].fillna('None')


df = df.dropna()

scaler = sk.preprocessing.StandardScaler()
df[['Number_of_Vehicles', 'Number_of_Casualties']] = scaler.fit_transform(df[['Number_of_Vehicles', 'Number_of_Casualties']])

df = pd.get_dummies(df, columns = ['Day_of_Week', 'Time', 'Speed_limit', 'Junction_Control', 'Pedestrian_Crossing-Human_Control',
                                   'Road_Surface_Conditions', 'Pedestrian_Crossing-Physical_Facilities',
                                'Light_Conditions', 'Weather_Conditions', 'Road_Surface_Conditions', 'Special_Conditions_at_Site', 
                                'Carriageway_Hazards', 'Urban_or_Rural_Area',
                               'Did_Police_Officer_Attend_Scene_of_Accident', 'Year'])

x_train, x_test, y_train, y_test = sk.model_selection.train_test_split(df.drop(columns = ['Accident_Severity']), df['Accident_Severity'], test_size = 0.2, random_state = 42)

              precision    recall  f1-score        support
1              0.024024  0.346302  0.044930    3881.000000
2              0.161518  0.100566  0.123954   40809.000000
3              0.866390  0.741934  0.799347  255233.000000
accuracy       0.649547  0.649547  0.649547       0.649547
macro avg      0.350644  0.396267  0.322744  299923.000000
weighted avg   0.759581  0.649547  0.697687  299923.000000


In [ ]:
naive_bayes = sk.naive_bayes.GaussianNB()
naive_bayes.fit(x_train, y_train)
y_hat = naive_bayes.predict(x_test)
report = sk.metrics.classification_report(y_test, y_hat, output_dict=True)
report = pd.DataFrame(report).transpose()
print(report)